In [ ]:
! pip install awswrangler

## 1. Upload data to S3

First you need to create a bucket for this experiment. Upload the data from the following public location to your own S3 bucket.

You can create a bucket from the following link: <a href='https://s3.console.aws.amazon.com/s3/home?region=us-east-1'> s3 console </a>

To facilitate the work of the crawler we will use two different prefixes (folders): one for the billing information and one for reseller. 



### Download the data

In [ ]:
# your bucket name
your_bucket = 'XXXXXXXXX'

In [ ]:
!wget https://ml-lab-mggaska.s3.amazonaws.com/billing_sm.csv
!wget https://ml-lab-mggaska.s3.amazonaws.com/reseller_sm.csv
!wget https://ml-lab-mggaska.s3.amazonaws.com/awswrangler-1.9.6-py3.6.egg

In [ ]:
import boto3, os
import awswrangler

In [ ]:
boto3.Session().resource('s3').Bucket(your_bucket).Object(os.path.join('billing', 'billing_sm.csv')).upload_file('billing_sm.csv')
boto3.Session().resource('s3').Bucket(your_bucket).Object(os.path.join('reseller', 'reseller_sm.csv')).upload_file('reseller_sm.csv')
boto3.Session().resource('s3').Bucket(your_bucket).Object(os.path.join('python', 'awswrangler-1.9.6-py3.6.egg')).upload_file('awswrangler-1.9.6-py3.6.egg')


## 2. Add athena full access permissions to SageMaker

In [ ]:
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

Go to the <a href='https://console.aws.amazon.com/iam/home?region=us-east-1#/roles'>IAM roles console</a> and attach the Amazon Athena full access policy to this role.

## 2. Create a Crawler

To use this csv information in the context of a Glue ETL, first we have to create a Glue crawler pointing to the location of each file. The crawler will try to figure out the data types of each column. 


1. On the <a href='https://console.aws.amazon.com/iam/home?region=us-east-1#/roles'>IAM roles console</a> create an IAM role GlueCrawlerRole with the policy AWSGlueServiceRole and S3FullAccess.

2. Go to  <a href='https://console.aws.amazon.com/glue/home?region=us-east-1#catalog:tab=crawlers'>Glue crawlers console</a> 

3. Add a Crawler : create one pointing to different each S3 locations (one to billing and one to reseller)

    3.1 Fill  a Crawler Name: point a Data Store to specific S3 path, Navigate to your bucket and your folder: /billing, click "Next"
    
    3.2 Specify "Yes" to add a new Data Store and navigate to your bucket and your folder: /reseller, Click "Next" and select "No" when asking for add more Data stores, use an existing IAM role "AWSGlueServiceRole", add database "implementationdb", Click on "Next" and "Finish"
    
    3.3 After the crawler is created select "Run it now".
    

## 3. Configure Athena query destination

Go to the <a href='https://console.aws.amazon.com/athena/home?force&region=us-east-1#query'>Athena console</a>.

Under Settings in the top right corner set the query results location to s3://YOUR-BUCKET-NAME/athena-results/.

To verify that your crawlers created correctly you can run the following query:
    
    select * from billing limit 3; 


## 4. Execute a query to create a sample View in Athena

In [ ]:

session = awswrangler.Session()
query=('CREATE VIEW resellers_sample AS SELECT *'
       'FROM billing where id_reseller '
       'in (select distinct id_reseller from reseller TABLESAMPLE BERNOULLI(10))')

df = session.pandas.read_sql_athena(
    sql=query,
    database="implementationdb",
    max_result_size=1
)